<a href="https://colab.research.google.com/github/fasghq/TT/blob/master/TT_Lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flax

     |████████████████████████████████| 143kB 3.2MB/s 


In [ ]:
from jax import ops
import jax.numpy as jnp
from jax.numpy import maximum
from jax import grad
import jax.numpy as np
from jax.numpy import linalg as la
from jax import jit
from jax import lax
import numpy as nnp
import math
import time
import flax
from flax import struct
import operator
import functools as ft
import copy

In [ ]:
class TT_Tensor:
    def __init__(self, core, d, n, r):
        self.core = core
        self.d = d
        self.n = n
        self.r = r

In [ ]:
@struct.dataclass
class Model():
    core: list

In [ ]:
def mod_to_old(ttm):
    d = len(ttm.core)
    n = []
    r = []
    for X in ttm.core:
        n.append(X.shape[1])
        r.append(X.shape[0])
    r.append(ttm.core[d - 1].shape[2])
    return TT_Tensor(ttm.core, d, n, r)

In [ ]:
def tt_get_d(tt):
  return len(tt.core)

In [ ]:
def tt_get_n(tt):
  d = len(tt.core)
  n = []
  for X in tt.core:
        n.append(X.shape[1])
  return n

In [ ]:
def tt_get_r(tt, *args):
    varargin = args
    nargin = 1 + len(varargin)
    d = len(tt.core)
    r = []
    for X in tt.core:
      r.append(X.shape[0])
    if nargin == 2:
      if varargin == "m":
        r.append(tt.core[d - 1].shape[3])
    else:
      r.append(tt.core[d - 1].shape[2])
    return r

In [ ]:
def tt_to_tensor(tt, shape):
    G = tt.core
    B = G[0]
    for i in range(1, len(G)):
        X = np.reshape(G[i], (G[i].shape[0], G[i].shape[1] * G[i].shape[2]))
        B = np.reshape(B, (int(np.size(B) / G[i].shape[0]), G[i].shape[0]))
        B = np.dot(B, X)
    B = np.reshape(B, shape)
    return B

In [ ]:
def tt_svd(A, delta):

    #time1 = time.perf_counter()

    d = len(A.shape)
    N = np.size(A)
    n = A.shape

    C = A
    G = [] 
    r = [] 
    rk1 = 1

    for k in range(1, d):
        C = np.reshape(C, (rk1 * n[k-1], int(N / (rk1 * n[k-1]))))      
        u, s, v = la.svd(C, full_matrices=False)
        rc = s.shape[0]  - (s < delta).sum()
        r.append(rc)
        G.append(np.reshape(u, (rk1, n[k-1], u.shape[1])))
        s = np.diag(s)
        C = np.dot(s, v)
        rk = C.shape[0]
        N = (N * rk) / (n[k-1] * rk1)
        rk1 = rk
    
    if len(C.shape) == 2:
        C = np.reshape(C, (C.shape[0], C.shape[1], 1))
        r.append(C.shape[2])
        
    G.append(C)

    #time2 = time.perf_counter()
    #print("\n time: ", time2 - time1)
    
    return Model(G), r

In [ ]:
def tt_sum_m(tt1, tt2):
    d1 = len(tt1.core)
    d2 = len(tt2.core)
    if d1 == d2:
        d = d1
    else:
        raise Exception("Different dimensions of tensors.")

    r1 = []
    n1 = []
    for X in tt1.core:
        n1.append(X.shape[1])
        n1.append(X.shape[2])
        r1.append(X.shape[0])
        r1.append(X.shape[3])

    r2 = []
    n2 = []
    for X in tt2.core:
        n2.append(X.shape[1])
        n2.append(X.shape[2])
        r2.append(X.shape[0])
        r2.append(X.shape[3])
    
    r = [x + y for x, y in zip(r1, r2)]
    
    if r1[0] == r2[0]:
        r[0] = r1[0]
    else:
        raise Exception("Different sizes of first mode.")
        
    if r1[len(r1)-1] == r2[len(r2)-1]:
        r[len(r1)-1] = r1[len(r1)-1]
    else:
        raise Exception("Different sizes of last mode.")
        
    n = n1
    G = []
    

    j = 0
    for i in range(0, d):
        Gi = nnp.zeros((r[j], n[j], n[j+1], r[j+1]))
        Gi = ops.index_update(Gi, ops.index[0 : r1[j], :, :, 0 : r1[j+1]], tt1.core[i])
        Gi = ops.index_update(Gi, ops.index[r[j] - r2[j] : r[j], :, :, r[j+1] - r2[j+1]: r[j+1]], tt2.core[i])
        G.append(Gi)
        j = j + 2
    
    return Model(G)

In [ ]:
def tt_sum(tt1, tt2):
    #time1 = time.perf_counter()
    d1 = len(tt1.core)
    d2 = len(tt2.core)
    if d1 == d2:
        d = d1
    else:
        raise Exception("Different dimensions of tensors.")

    r1 = []
    n1 = []
    for X in tt1.core:
        n1.append(X.shape[1])
        r1.append(X.shape[0])
    r1.append(tt1.core[len(tt1.core) - 1].shape[2])

    r2 = []
    n2 = []
    for X in tt2.core:
        n2.append(X.shape[1])
        r2.append(X.shape[0])
    r2.append(tt2.core[len(tt2.core) - 1].shape[2])
    
    r = [x + y for x, y in zip(r1, r2)]
    
    if r1[0] == r2[0]:
        r[0] = r1[0]
    else:
        raise Exception("Different sizes of first mode.")
        
    if r1[d] == r2[d]:
        r[d] = r1[d]
    else:
        raise Exception("Different sizes of last mode.")
        
    n = n1
    G = []
    
    print(r1, r2, r)

    for i in range(0, d):
        print(i)
        Gi = nnp.zeros((r[i], n[i], r[i+1]))
        Gi = ops.index_update(Gi, ops.index[0 : r1[i], :, 0 : r1[i+1]], tt1.core[i])
        Gi = ops.index_update(Gi, ops.index[r[i] - r2[i] : r[i], :, r[i+1] - r2[i+1]: r[i+1]], tt2.core[i])
        #Gi[0 : r1[i], :, 0 : r1[i+1]] = tt1.core[i]
        #Gi[r[i] - r2[i] : r[i], :, r[i+1] - r2[i+1]: r[i+1]] = tt2.core[i]   
        G.append(Gi)

    #time2 = time.perf_counter()
    #print("\n time: ", time2 - time1)
    
    return Model(G)

In [ ]:
def tt_round(tt, delta):  
    G = tt.core
    d = len(tt.core)
    rn = []
    #time1 = time.perf_counter()    
    
    for k in range(d-1, 0, -1):
        r1, n, r2 = G[k].shape
        G[k] = np.reshape(G[k], (r1, n * r2))
        G[k] = np.transpose(G[k])
        G[k], R = la.qr(G[k])
        G[k] = np.transpose(G[k])
        G[k-1] = np.einsum('ijk,lk->ijl', G[k-1], R) 
    
    for k in range(0, d - 1):
        
        if k == 0:
            r1, n1, r2 = G[k].shape
            r3 = G[k+2].shape[0]
            n2 = int(G[k+1].shape[1] / r3)
        elif k == d - 2:
            r1 = G[k].shape[0]
            r2 = G[k+1].shape[0]
            n1 = int(G[k].shape[1] / r2)
            r3 = 1
            n2 = G[k+1].shape[1]
        else:
            r1 = G[k].shape[0]
            r2 = G[k+1].shape[0]
            n1 = int(G[k].shape[1] / r2)
            r3 = G[k+2].shape[0]
            n2 = int(G[k+1].shape[1] / r3)

        G[k] = np.reshape(G[k], (r1 * n1, r2))

        u, s, v = la.svd(G[k], full_matrices=False)
 
        rc = s.shape[0]  - (s < delta).sum()
        rn.append(rc)
        s = np.diag(s)  
         
        v = np.dot(s, v) 
        G[k] = np.reshape(u, (r1, n1, u.shape[1]))
        G[k+1] = np.einsum('ij,ik->kj', G[k+1], v.T)
    
    
    if len(G[d-1].shape) == 2:
        G[d-1] = np.reshape(G[d-1], (G[d-1].shape[0], G[d-1].shape[1], 1))

    #time2 = time.perf_counter()
    #print("\n time: ", time2 - time1)
    
    return Model(G), rn

In [ ]:
def tt_matrix(A):
  '''
  dim = 0
  a = np.prod(A.size)
  while a % 2 == 0:
    a = int(a / 2)
    dim = dim + 1

  A = A.reshape([2 for i in range(dim)])
  '''
  d = int(len(A.shape) / 2)
  nper = []
  for i in range(d):
    nper.append(i)
    nper.append(i+d)
  return A.transpose(nper)

In [ ]:
def full_matrix(A):
  d = int(len(A.shape))
  d2 = int(len(A.shape) / 2)
  nper = [*range(0, d, 2)] + [*range(1, d, 2)]
  return A.transpose(nper).reshape(ft.reduce(operator.mul, A.shape[:d2], 1), ft.reduce(operator.mul, A.shape[d2:], 1))
  '''
  dim = 0
  a = np.prod(A.size)
  while a % 4 == 0:
    a = int(a / 4)
    dim = dim + 1
  return A.transpose(nper).reshape([4 for i in range(dim)])
  '''

In [ ]:
def tt_matvec(ttm, ttv):
  d = len(tt.core)
  M = ttm.core
  x = ttv.core
  y = []
  for k in range(d):
    mshp = M[k].shape
    vshp = x[k].shape
    M[k] = np.reshape(M[k], (ft.reduce(operator.mul, mshp[:3], 1), mshp[3]))
    M[k] = M[k].T
    M[k] = np.reshape(M[k], (mshp[3] * mshp[0] * mshp[1], mshp[2]))
    x[k] = x[k].transpose([1,0,2])
    x[k] = np.reshape(x[k], (vshp[1], vshp[0] * vshp[2]))
    y.append(M[k].dot(x[k]))
    y[k] = np.reshape(y[k], (mshp[3], mshp[0], mshp[1], vshp[0], vshp[2]))
    y[k] = y[k].transpose([1,3,2,0,4])
    shp = y[k].shape
    y[k] = np.reshape(y[k], (shp[0] * shp[1], shp[2], shp[3] * shp[4]))
  return Model(y)



In [ ]:
def tt_kron_prod(tt1, tt2):
  d = len(tt1.core)
  cr1 = tt1.core
  cr2 = tt2.core
  cr3 = []
  for i in range(d):
    shp1 = cr1[i].shape
    shp2 = cr2[i].shape
    cr1[i] = np.reshape(cr1[i], (shp1[0] * shp1[1], shp1[2] * shp1[3]))
    cr2[i] = np.reshape(cr2[i], (shp2[0] * shp2[1], shp2[2] * shp2[3]))
    cr3.append(np.kron(cr1[i], cr2[i]))
    cr3[i] = np.reshape(cr3[i], (shp1[0], shp1[1], shp2[0], shp2[1], shp1[2], shp1[3], shp2[2], shp2[3]))
    #cr3[i] = cr3[i].transpose([0, 2, 3, 1, 6, 4, 5, 7])
  return Model(cr3)

In [ ]:
def laplace(dim):
  eps = 1e-2

  #prepare I matrix
  I = np.eye(2**dim)
  I = np.asarray(I)
  print("I:", I, I.shape)

  dn = 0
  a = np.prod(I.size)
  while a % 2 == 0:
    a = int(a / 2)
    dn = dn + 1
  I = I.reshape([2 for i in range(dn)])

  I = tt_matrix(I)
  shp = I.shape

  dn = 0
  a = np.prod(I.size)
  while a % 4 == 0:
    a = int(a / 4)
    dn = dn + 1
  I = I.reshape([4 for i in range(dn)])

  fast_tt_svd = jit(tt_svd, static_argnums=(1))
  tti, rn = fast_tt_svd(I, eps / math.sqrt(len(I.shape) - 1))

  for i in range(len(rn)-1):  
    tti.core[i] = tti.core[i][:, :, :rn[i]]
    tti.core[i+1] = tti.core[i+1][:rn[i], :, :]

  ptr = 0
  for i in range(len(tti.core)):
    tti.core[i] = np.reshape(tti.core[i], (tti.core[i].shape[0], shp[ptr], shp[ptr+1], tti.core[i].shape[2]))
    ptr = ptr + 2

  #prepare L matrix
  md = 2 * np.ones((1, 2**dim))[0]
  uld = -np.ones((1, 2**dim - 1))[0]
  L = np.diag(md, 0) + np.diag(uld, -1) + np.diag(uld, 1)
  L = nnp.asarray(L)
  print("L:", L, L.shape)

  dn = 0
  a = np.prod(L.size)
  while a % 2 == 0:
    a = int(a / 2)
    dn = dn + 1
  L = L.reshape([2 for i in range(dn)])

  L = tt_matrix(L)
  shp = L.shape

  dn = 0
  a = np.prod(L.size)
  while a % 4 == 0:
    a = int(a / 4)
    dn = dn + 1
  L = L.reshape([4 for i in range(dn)])

  fast_tt_svd = jit(tt_svd, static_argnums=(1))
  ttl, rm = fast_tt_svd(L, eps / math.sqrt(len(L.shape) - 1))

  for i in range(len(rn)-1):  
    ttl.core[i] = ttl.core[i][:, :, :rm[i]]
    ttl.core[i+1] = ttl.core[i+1][:rm[i], :, :]

  ptr = 0
  for i in range(len(ttl.core)):
    ttl.core[i] = np.reshape(ttl.core[i], (ttl.core[i].shape[0], shp[ptr], shp[ptr+1], ttl.core[i].shape[2]))
    ptr = ptr + 2

  #calculate LxI & IxL 
  cr1 = []
  cr2 = []
  
  for i in range(len(ttl.core)):
    cr1.append(ttl.core[i])
    cr1.append(tti.core[i])
    cr2.append(tti.core[i])
    cr2.append(ttl.core[i])

  tt1 = Model(cr1)
  tt2 = Model(cr2)

  print("I ranks:")
  for X in tti.core:
    print(X.shape)
  print("L ranks:")
  for X in ttl.core:
    print(X.shape)
  print("LxI ranks:")
  for X in tt1.core:
    print(X.shape)
  print("IxL ranks:")
  for X in tt2.core:
    print(X.shape)

  tt = tt_sum_m(tt1, tt2)

  return tti, ttl, tt1, tt2, tt


In [ ]:
tti, ttl, tt1, tt2, tt = laplace(6)
print("I ranks:")
for X in tti.core:
  print(X.shape)
print("L ranks:")
for X in ttl.core:
  print(X.shape)
print("LxI ranks:")
for X in tt1.core:
  print(X.shape)
print("IxL ranks:")
for X in tt2.core:
  print(X.shape)
print("tt ranks:")
for X in tt.core:
  print(X.shape)


I: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]] (64, 64)
L: [[ 2. -1.  0. ...  0.  0.  0.]
 [-1.  2. -1. ...  0.  0.  0.]
 [ 0. -1.  2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  2. -1.  0.]
 [ 0.  0.  0. ... -1.  2. -1.]
 [ 0.  0.  0. ...  0. -1.  2.]] (64, 64)
I ranks:
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
L ranks:
(1, 2, 2, 3)
(3, 2, 2, 3)
(3, 2, 2, 3)
(3, 2, 2, 3)
(3, 2, 2, 3)
(3, 2, 2, 1)
LxI ranks:
(1, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 1)
(1, 2, 2, 1)
IxL ranks:
(1, 2, 2, 1)
(1, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 3)
(1, 2, 2, 1)
(3, 2, 2, 1)
I ranks:
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
(1, 2, 2, 1)
L ranks:
(1, 2, 2, 3)
(3, 2, 2, 3

In [ ]:
'''
A = []
dim = 2
x = 0
for i in range(2 ** dim):
  A.append(math.sin(x))
  x = x + math.pi / (2 ** dim - 1)
shp = [2 for i in range(dim)]
A = np.asarray(A)
A = A.reshape(shp)
'''

dim = 6
#A = np.arange(2**(dim))
A = np.eye(2**(dim))
A = np.asarray(A)
eps = 1e-2 

print(A, "def")

dim = 0
a = np.prod(A.size)
while a % 2 == 0:
  a = int(a / 2)
  dim = dim + 1
A = A.reshape([2 for i in range(dim)])

#print(A, "\n\n", A.shape, "old\n")
print(A.shape, "old\n")

A = tt_matrix(A)
shp = A.shape


#print(A, "\n\n", A.shape, "new")
print(A.shape, "new\n")

B = full_matrix(A)
print(B.shape, "new2")
#print(B, "\n\n", B.shape, "new2")

dim = 0
a = np.prod(A.size)
while a % 4 == 0:
  a = int(a / 4)
  dim = dim + 1
A = A.reshape([4 for i in range(dim)])

fast_tt_svd = jit(tt_svd, static_argnums=(1))
tt, rn = fast_tt_svd(A, eps / math.sqrt(len(A.shape) - 1))

for i in range(len(rn)-1):  
    tt.core[i] = tt.core[i][:, :, :rn[i]]
    tt.core[i+1] = tt.core[i+1][:rn[i], :, :]

ptr = 0
for i in range(len(tt.core)):
  tt.core[i] = np.reshape(tt.core[i], (tt.core[i].shape[0], shp[ptr], shp[ptr+1], tt.core[i].shape[2]))
  ptr = ptr + 2
  
for X in tt.core:
  print("c:\n", X, X.shape)

print(tt_get_r(tt, "m"))

C = tt_to_tensor(tt, A.shape)
#print(C)

D = full_matrix(C)
#print(D, "\n\n", D.shape, "new3")

dimv = 6
vec = np.arange(2**(dimv))
vec = np.asarray(vec)
vec = vec.reshape([2 for i in range(dimv)])
print(vec.shape, "vec")

ttv, rn = fast_tt_svd(vec, eps / math.sqrt(len(vec.shape) - 1))

for i in range(len(rn)-1):  
    ttv.core[i] = ttv.core[i][:, :, :rn[i]]
    ttv.core[i+1] = ttv.core[i+1][:rn[i], :, :]
  
for X in ttv.core:
  print("c:\n", X, X.shape)

print(tt_get_r(ttv))

ttmv = tt_matvec(tt, ttv)
  
for X in ttmv.core:
  print("c:\n", X, X.shape)

print(tt_get_r(ttmv))


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]] def
(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2) old

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2) new

(64, 64) new2
c:
 [[[[-0.70710677]
   [-0.        ]]

  [[-0.        ]
   [-0.70710677]]]] (1, 2, 2, 1)
c:
 [[[[ 7.0710671e-01]
   [-5.5134647e-22]]

  [[ 0.0000000e+00]
   [ 7.0710677e-01]]]] (1, 2, 2, 1)
c:
 [[[[-0.7071069 ]
   [ 0.        ]]

  [[ 0.        ]
   [-0.70710677]]]] (1, 2, 2, 1)
c:
 [[[[-7.0710683e-01]
   [-7.0188613e-30]]

  [[-2.3179482e-22]
   [-7.0710677e-01]]]] (1, 2, 2, 1)
c:
 [[[[-7.0710677e-01]
   [-1.2690109e-29]]

  [[ 8.3096708e-23]
   [-7.0710677e-01]]]] (1, 2, 2, 1)
c:
 [[[[5.656854]
   [0.      ]]

  [[0.      ]
   [5.656853]]]] (1, 2, 2, 1)
[1, 1, 1, 1, 1, 1]
(2, 2, 2, 2, 2, 2) vec
c:
 [[[-0.33528185  0.94211787]
  [-0.94211787 -0.33528185]]] (1, 2, 2)
c:
 [[[ 0.5506017  -0.32400337]
  [ 0.829273    0.17468733]]

 [

In [ ]:
# input tensor A
'''
A = np.array([[1/2, 1/3, 1/4], [1/3, 1/4, 1/5], [1/4, 1/5, 1/6]])
A = np.arange(24)
B = A.reshape(4, 3, 2)
A = B
'''

'''
A = np.ones(1000000)
A = A.reshape(100, 100, 100)
#4 3 2
'''
'''
A = []
for i in range(100):
  for j in range(100):
    for q in range(100):
      A.append(1 / (i + j + q + 3))
A = np.asarray(A)
A = A.reshape(100, 100, 100)
'''

A = []
dim = 15
x = 0
for i in range(2 ** dim):
  A.append(math.sin(x))
  x = x + math.pi / (2 ** dim - 1)
shp = [2 for i in range(dim)]
A = np.asarray(A)
A = A.reshape(shp)

eps = 1e-2 # accuracy


fast_tt_svd = jit(tt_svd, static_argnums=(1))
tt, rn = fast_tt_svd(A, eps / math.sqrt(len(A.shape) - 1))
#ttnew, rn = tt_svd(A, eps / math.sqrt(len(A.shape) - 1))

for i in range(len(rn)-1):  
    tt.core[i] = tt.core[i][:, :, :rn[i]]
    tt.core[i+1] = tt.core[i+1][:rn[i], :, :]

fast_tt_sum = jit(tt_sum)
%timeit tts = fast_tt_sum(tt, tt)
#%timeit tts = tt_sum(tt, tt)
#for X in tts.core:
#  print(X)

ttsold = mod_to_old(tts)
print("\nRanks tts: ", ttsold.r)

fast_tt_round = jit(tt_round, static_argnums=(1))
ttr, rn = fast_tt_round(tts, eps / math.sqrt(len(tts.core) - 1))
#ttr, rn = tt_round(tts, eps / math.sqrt(len(tts.core) - 1))

for i in range(len(rn)):  
    ttr.core[i] = ttr.core[i][:, :, :rn[i]]
    ttr.core[i+1] = ttr.core[i+1][:rn[i], :, :]


#ttr2 = ttpy.matrix.round(tts)
#ttr2 = ttpy.tensor.round(tts, eps)
'''
print("\nttr:")
for X in ttr.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)
'''
ttro = mod_to_old(ttr)
print("\nRanks ttr: ", ttro.r)
'''
B = tt_to_tensor(tts, A.shape)
#print("\nB = A + A:\n", B)
C = tt_to_tensor(ttro, A.shape)
#print("\nC = round(B):\n", C)
print("\n", la.norm(B-C), " <= ", eps*la.norm(B), la.norm(B-C) <= eps*la.norm(B))
'''


The slowest run took 199.50 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 870 µs per loop

Ranks tts:  [1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1]

Ranks ttr:  [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


'\nB = tt_to_tensor(tts, A.shape)\n#print("\nB = A + A:\n", B)\nC = tt_to_tensor(ttro, A.shape)\n#print("\nC = round(B):\n", C)\nprint("\n", la.norm(B-C), " <= ", eps*la.norm(B), la.norm(B-C) <= eps*la.norm(B))\n'